In [1]:
from arb_via_load_directory import definir_diretorio
project_path = definir_diretorio()

import geopandas as gpd
from os.path import join
from unidecode import unidecode

from utils import (
    save_parquet_excel,
    check_crs,
    diretorio,
    get_data_diretorio,
)
from utils.downloads import (
    _prepare_cache, 
    download_malha_geosampa,
)

In [2]:
data_path = get_data_diretorio(project_path)

# Declarar variáveis

In [3]:
gdf_distritos_ibge = gpd.GeoDataFrame()
gdf_distritos = gpd.GeoDataFrame()

# IBGE

## Importar dados 

Inicialmente, a malha dos distritos vinha do geosampa. Contudo, por conta das incompatibilidades das malhas e, principalmente, como a malha do IBGE já vem com a informação dos km^2 e população, optei por utilizá-la.

(Porém, ao final deste notebook ainda há o carregamento da malha do geosampa)

In [4]:
cache_path = join(
    data_path, 
    'cache'
)

In [5]:
gdf_distritos_ibge = _prepare_cache(
    url='http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/shp/UF/SP/SP_distritos_CD2022.zip',
    name_feature= 'distritos_ibge',
    cache_path=cache_path
)

No caso dos distritos do IBGE, é preciso pegar apenas os da cidade de São Paulo, já que vêm os do Estado inteiro.

In [6]:

gdf_distritos_ibge = (
    gdf_distritos_ibge
    .loc[gdf_distritos_ibge['CD_MUN']=='3550308']
)

## Padronizaçã de colunas

### Nome das colunas

Originalmente, muitas das colunas dos gdfs que serão interseccionados têm nomes repetidos, ou estavam vazias. Para que isso não faça com que a análise das tabelas seja confusa, apagaremos as colunas que julgadas desnecessárias, e renomearemos as necessárias.

In [7]:
gdf_distritos_ibge.columns

Index(['CD_REGIAO', 'NM_REGIAO', 'CD_UF', 'NM_UF', 'CD_MUN', 'NM_MUN',
       'CD_DIST', 'NM_DIST', 'CD_RGINT', 'NM_RGINT', 'CD_RGI', 'NM_RGI',
       'CD_CONCURB', 'NM_CONCURB', 'AREA_KM2', 'v0001', 'v0002', 'v0003',
       'v0004', 'v0005', 'v0006', 'v0007', 'geometry'],
      dtype='object')

In [8]:
drop_distritos_cols={
    'CD_REGIAO',
    'NM_REGIAO',
    'CD_UF',
    'NM_UF',
}

gdf_distritos_ibge=gdf_distritos_ibge.drop(columns=drop_distritos_cols)

In [9]:
drop_cols = [f'v000{i}' for i in range(3,8)]
gdf_distritos_ibge = gdf_distritos_ibge.drop(columns=drop_cols)

In [10]:
rename_distrito_ibge_cols={
    'v0001' : 'total_pop', 
    'v0002' : 'total_dom'
}

gdf_distritos_ibge.rename(rename_distrito_ibge_cols, axis=1, inplace=True)

### Conteúdo das colunas

Quando formos realizar operações entre distritos e outras malhas, caso a outra malha não tenha a informação do CD_DIST, as comparações precisarão ser feitas a partir da coluna 'NM_DIST'. Portante seu conteúdo deve ser padronizado também:

In [11]:
gdf_distritos_ibge['NM_DIST']=gdf_distritos_ibge['NM_DIST'].str.upper()


In [12]:
for value in gdf_distritos_ibge['NM_DIST']:
    (
        gdf_distritos_ibge
        .loc[gdf_distritos_ibge['NM_DIST'] == value, 'NM_DIST']
    ) = unidecode(value)

Alguns dos valores que precisam ser inteiros estão constando como str e precisam ser padronizados também:

In [13]:
gdf_distritos_ibge['total_pop'] = (
    gdf_distritos_ibge['total_pop']
    .astype(int)
)

## Conferir CRS

In [14]:
gdf_distritos_ibge = check_crs(gdf_distritos_ibge)

O crs do gdf não está correto:EPSG:4674


## Salvar GDF

In [15]:
save_parquet_excel(
    gdf_distritos_ibge,
    'distrito_ibge',
    data_path,
    'assets'
)

